# Enhanced Feature Engineering and Model Evaluation

## Step 1: Load the Dataset

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the enhanced data
df = pd.read_csv('/mnt/data/enhanced_data.csv')
df.head()


## Step 2: Remove Irrelevant Columns

In [ ]:

# Remove irrelevant columns
irrelevant_columns = ['id', 'date_end', 'date_modif_prod', 'date_renewal']
df.drop(columns=irrelevant_columns, inplace=True)
df.head()


## Step 3: Expand the Dataset with New Features

In [ ]:

# Extract components from date columns (if not already extracted)
df['year_activ'] = pd.to_datetime(df['date_activ']).dt.year
df['month_activ'] = pd.to_datetime(df['date_activ']).dt.month
df['day_activ'] = pd.to_datetime(df['date_activ']).dt.day
df['day_of_week_activ'] = pd.to_datetime(df['date_activ']).dt.dayofweek
df['quarter_activ'] = pd.to_datetime(df['date_activ']).dt.quarter
df.drop(columns=['date_activ'], inplace=True)
df.head()


## Step 4: Combine Columns to Create Better Features

In [ ]:

# Create a feature for total consumption by summing consumption columns
df['total_consumption'] = df['cons_12m'] + df['cons_gas_12m'] + df['cons_last_month']

# Create the new feature 'price_diff_dec_jan'
df['price_off_peak_dec'] = df['var_6m_price_off_peak'] * 1.02  # Dummy calculation for December prices
df['price_off_peak_jan'] = df['var_6m_price_off_peak'] * 0.98  # Dummy calculation for January prices
df['price_diff_dec_jan'] = df['price_off_peak_dec'] - df['price_off_peak_jan']

# Create interaction features
df['price_consumption_interaction'] = df['total_consumption'] * df['price_diff_dec_jan']

df[['price_off_peak_dec', 'price_off_peak_jan', 'price_diff_dec_jan', 'price_consumption_interaction']].head()


## Step 5: Prepare the Final Dataset

In [ ]:

# Prepare the dataset
features = df.drop(columns=['churn'])
target = df['churn']

# Identify categorical columns
categorical_cols = features.select_dtypes(include=['object']).columns

# Define preprocessing pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing for numeric and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'
)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)


## Step 6: Train and Evaluate the Model Without the New Feature

In [ ]:

# Exclude the new feature from training
X_train_without_new = X_train.drop(columns=['price_diff_dec_jan'])
X_test_without_new = X_test.drop(columns=['price_diff_dec_jan'])

# Define the model
model = RandomForestClassifier(random_state=42)

# Create and evaluate pipeline without the new feature
pipeline_without_new = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Train the model without the new feature
pipeline_without_new.fit(X_train_without_new, y_train)
predictions_without_new = pipeline_without_new.predict(X_test_without_new)
accuracy_without_new = accuracy_score(y_test, predictions_without_new)
roc_auc_without_new = roc_auc_score(y_test, predictions_without_new)

accuracy_without_new, roc_auc_without_new


## Step 7: Train and Evaluate the Model With the New Feature

In [ ]:

# Create and evaluate pipeline with the new feature
pipeline_with_new = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Train the model with the new feature
pipeline_with_new.fit(X_train, y_train)
predictions_with_new = pipeline_with_new.predict(X_test)
accuracy_with_new = accuracy_score(y_test, predictions_with_new)
roc_auc_with_new = roc_auc_score(y_test, predictions_with_new)

accuracy_with_new, roc_auc_with_new


## Step 8: Compare the Performance of Both Models

In [ ]:

print(f'Accuracy without new feature: {accuracy_without_new}')
print(f'ROC AUC without new feature: {roc_auc_without_new}')
print(f'Accuracy with new feature: {accuracy_with_new}')
print(f'ROC AUC with new feature: {roc_auc_with_new}')


## Step 9: Visualize the Performance Comparison

In [ ]:

import matplotlib.pyplot as plt

# Plotting function to compare results
def plot_comparison(title, y_label, values_without_new, values_with_new, labels):
    x = range(len(values_without_new))
    width = 0.4

    fig, ax = plt.subplots()
    ax.bar(x, values_without_new, width, label='Without New Feature')
    ax.bar([p + width for p in x], values_with_new, width, label='With New Feature')

    ax.set_xlabel('Metrics')
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.set_xticks([p + width/2 for p in x])
    ax.set_xticklabels(labels)
    ax.legend()

    plt.show()

# Values for plotting
metrics_labels = ['Accuracy', 'ROC AUC']
values_without_new = [accuracy_without_new, roc_auc_without_new]
values_with_new = [accuracy_with_new, roc_auc_with_new]

# Plot the comparison
plot_comparison('Model Performance Comparison', 'Score', values_without_new, values_with_new, metrics_labels)
